In [3]:
lst = [1,2,2,2,3,4,4,4,5,6,7]
dup = {}

for item in lst:
  if item in dup:
    dup[item] += 1
  else:
    dup[item] = 1
for key in dup:
  # if dup[key] > 1:
    print(key, 'is',dup[key] )

1 is 1
2 is 3
3 is 1
4 is 3
5 is 1
6 is 1
7 is 1


In [4]:
words = ['qqwsw','eerdece','e','frgfrg']
resut = ''
for word in words:
  resut += word + str(len(word))
print(resut)

qqwsw5eerdece7e1frgfrg6


In [18]:
num = 121
strnum = str(num)
trnum = strnum[::-1]

if trnum == strnum:
  print('yes')
else:
  print('no')

yes


In [20]:
num = 121
strnum = str(num)
revnum = strnum[::-1]

if strnum == revnum:
    print('yes')
else:
    print('no')

yes


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
# 1. Sample Data
data = [
    ("U001", "2024-01-01 10:00:00", 100),
    ("U001", "2024-02-01 09:00:00", 105),
    ("U002", "2024-01-05 11:00:00", 200),
    ("U002", "2024-01-05 08:00:00", 195),
]
columns = ["user_id", "last_updated", "value"]
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(data, columns)
# 2. Convert to timestamp if needed
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp
df = df.withColumn("last_updated", to_timestamp("last_updated"))
# 3. Define Window Spec
window_spec = Window.partitionBy("user_id").orderBy(col("last_updated").desc())
# 4. Assign Row Numbers
ranked_df = df.withColumn("row_num", row_number().over(window_spec))
# 5. Filter for Most Recent Record
final_df = ranked_df.filter(col("row_num") == 1).drop("row_num")
final_df.show()

+-------+-------------------+-----+
|user_id|       last_updated|value|
+-------+-------------------+-----+
|   U001|2024-02-01 09:00:00|  105|
|   U002|2024-01-05 11:00:00|  200|
+-------+-------------------+-----+



In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, sum, rank
from pyspark.sql.window import Window

# Start Spark session
spark = SparkSession.builder.appName("PySpark_Interview_Demo").getOrCreate()

# Sample DataFrame 1: Users
users_data = [
    (1, "Alice", "2024-01-01", "HR"),
    (2, "Bob", "2024-02-01", "Finance"),
    (3, "Charlie", "2024-01-15", "HR"),
    (4, "David", "2024-03-01", "IT"),
    (5, "Eve", "2024-01-10", "Finance"),
]
users_cols = ["user_id", "name", "join_date", "department"]

users_df = spark.createDataFrame(users_data, users_cols)

# Sample DataFrame 2: Salaries
salary_data = [
    (1, 1000),
    (2, 2000),
    (3, 1500),
    (4, 2200),
    (6, 1800)  # user_id 6 does not exist in users_df
]
salary_cols = ["user_id", "salary"]
salary_df = spark.createDataFrame(salary_data, salary_cols)

In [7]:
# Inner Join
inner_join = users_df.join(salary_df, "user_id", "inner")
# Left Join
left_join = users_df.join(salary_df, "user_id", "left")
# Left Anti Join (users not in salary)
left_anti = users_df.join(salary_df, "user_id", "left_anti")
# Left Semi Join (users with matching salary)
left_semi = users_df.join(salary_df, "user_id", "left_semi")

In [8]:
# Duplicate users for merge simulation
extra_users = [
    (6, "Frank", "2024-04-01", "IT"),
    (7, "Grace", "2024-04-10", "HR")
]
extra_users_df = spark.createDataFrame(extra_users, users_cols)

# Combine
merged_users = users_df.union(extra_users_df).dropDuplicates()


In [9]:
# Join salary with users first
user_salary_df = users_df.join(salary_df, "user_id", "left")

# Add join_date as timestamp
user_salary_df = user_salary_df.withColumn("join_date", col("join_date").cast("timestamp"))

# Define window by department
window_spec = Window.partitionBy("department").orderBy(col("join_date"))

# Add running sum
user_salary_df = user_salary_df.withColumn("running_total",
    sum("salary").over(window_spec)
)

# Add row number
user_salary_df = user_salary_df.withColumn("row_num",
    row_number().over(window_spec)
)


In [10]:
# Filter users who joined after Jan 15
filtered_df = users_df.filter(col("join_date") > "2024-01-15")

# Group by department and count
dept_count = users_df.groupBy("department").count()

# Sort by name
sorted_users = users_df.orderBy("name")


In [22]:
inner_join.show()
user_salary_df.select("user_id", "name", "department", "salary", "running_total", "row_num").show()
dept_count.show()

+-------+-------+----------+----------+------+
|user_id|   name| join_date|department|salary|
+-------+-------+----------+----------+------+
|      1|  Alice|2024-01-01|        HR|  1000|
|      2|    Bob|2024-02-01|   Finance|  2000|
|      3|Charlie|2024-01-15|        HR|  1500|
|      4|  David|2024-03-01|        IT|  2200|
+-------+-------+----------+----------+------+

+-------+-------+----------+------+-------------+-------+
|user_id|   name|department|salary|running_total|row_num|
+-------+-------+----------+------+-------------+-------+
|      5|    Eve|   Finance|  NULL|         NULL|      1|
|      2|    Bob|   Finance|  2000|         2000|      2|
|      1|  Alice|        HR|  1000|         1000|      1|
|      3|Charlie|        HR|  1500|         2500|      2|
|      4|  David|        IT|  2200|         2200|      1|
+-------+-------+----------+------+-------------+-------+

+----------+-----+
|department|count|
+----------+-----+
|        HR|    2|
|   Finance|    2|
|    

In [19]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Sample Data
data = [
    (1, "2024-01-01 09:00:00", "HR"),
    (1, "2024-01-03 10:00:00", "HR"),  # latest for user 1
    (2, "2024-01-02 14:00:00", "IT")
]

df = spark.createDataFrame(data, ["user_id", "last_updated", "dept"])

# Convert to timestamp
from pyspark.sql.functions import to_timestamp
df = df.withColumn("last_updated", to_timestamp("last_updated"))

# Define Window
window_spec = Window.partitionBy("user_id").orderBy(df["last_updated"].desc())

# Apply row_number and filter
from pyspark.sql.functions import col
df_ranked = df.withColumn("row_num", row_number().over(window_spec))
most_recent = df_ranked.filter(col("row_num") == 1).drop("row_num").show()

+-------+-------------------+----+
|user_id|       last_updated|dept|
+-------+-------------------+----+
|      1|2024-01-03 10:00:00|  HR|
|      2|2024-01-02 14:00:00|  IT|
+-------+-------------------+----+



In [ ]:
from pyspark.sql.functions import broadcast

# large_df: transaction data, millions of rows
# small_df: lookup table (e.g. department names), ~10 rows

joined_df = large_df.join(broadcast(small_df), on="dept_id", how="inner")

In [15]:
from pyspark.sql.functions import rank
from pyspark.sql.window import Window

# Sample data
data = [
    ("Alice", "HR", 1000),
    ("Bob", "HR", 1200),
    ("Charlie", "HR", 1200),
    ("David", "IT", 1500),
]
df = spark.createDataFrame(data, ["name", "dept", "salary"])

# Define window partitioned by department and ordered by salary descending
window_spec = Window.partitionBy("dept").orderBy(col("salary").desc())

# Add rank column
df_with_rank = df.withColumn("rank", rank().over(window_spec)).show()


+-------+----+------+----+
|   name|dept|salary|rank|
+-------+----+------+----+
|    Bob|  HR|  1200|   1|
|Charlie|  HR|  1200|   1|
|  Alice|  HR|  1000|   3|
|  David|  IT|  1500|   1|
+-------+----+------+----+



In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, to_timestamp
# Create Spark session
spark = SparkSession.builder.appName("LatestUserRecords").getOrCreate()
# Sample data
data = [
    (1, "login", "2024-05-20 10:00:00"),
    (1, "login", "2024-05-21 09:00:00"),
    (2, "logout", "2024-05-19 18:00:00"),
    (2, "logout", "2024-05-21 20:00:00")
]
columns = ["id", "act", "time"]
df = spark.createDataFrame(data, columns)
df = df.withColumn('time', to_timestamp('time'))
wf = Window.partitionBy('id').orderBy(col('time').desc())
df1 = df.withColumn('rn', row_number().over(wf))
dfdf = df1.filter(col('rn') == 1).drop('rn')
dfdf.show()

+---+------+-------------------+
| id|   act|               time|
+---+------+-------------------+
|  1| login|2024-05-21 09:00:00|
|  2|logout|2024-05-21 20:00:00|
+---+------+-------------------+

